### I Don't Wanna pay fucking fees using bot !

In [51]:
import json
import math
import time
import pytz
import secrets
import requests
import credentials
from web3 import Web3
from datetime import datetime
from eth_account import Account

### Connect to Ethereum

In [2]:
INFURA_KEY = credentials.infura_key
ETHERSCAN_KEY = credentials.etherscan_key

RPC = {
    'mainnet': f'https://mainnet.infura.io/v3/{INFURA_KEY}',
    'goreli': f'https://goerli.infura.io/v3/{INFURA_KEY}',
    'mainnet_ws': f'wss://mainnet.infura.io/ws/v3/{INFURA_KEY}',
    'goreli_ws': f'wss://goerli.infura.io/ws/v3/{INFURA_KEY}',
    'mainnet_mev': f'https://rpc.mevblocker.io'
}

# Mainnet MEV protector
# provider = RPC['mainnet_mev']
# w3 = Web3(Web3.HTTPProvider(provider, request_kwargs={'timeout': 60}))

provider = RPC['goreli_ws']
w3 = Web3(Web3.WebsocketProvider(provider))
w3.is_connected()

True

In [ ]:
from web3.middleware import geth_poa_middleware, simple_cache_middleware

provider = RPC['mainnet_mev']
w3 = Web3(Web3.HTTPProvider(provider, request_kwargs={'timeout': 60}))

w3.middleware_onion.add(simple_cache_middleware)
w3.middleware_onion.add(geth_poa_middleware)

# Callback function to handle new blocks
def handle_new_block(block_num, max_retries=3, delay=0.1):
    retries = 0
    while retries < max_retries:
        try:
            block = w3.eth.get_block(block_num)
            print(f"New Block: {block['number']}")
            return
        except:
            retries += 1
            print(f"Block not found. Retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception(f"Failed to retrieve block {event} after {max_retries} attempts.")

latest_block = w3.eth.block_number
while True:
    current_block = w3.eth.block_number
    if current_block > latest_block:
        for block_number in range(latest_block + 1, current_block + 1):
            handle_new_block(block_number)
        latest_block = current_block

### Helper functions

In [3]:
def get_abi(contract_address, chain_id=1):
    #Get Contract ABI with Etherscan API
    api_key = ETHERSCAN_KEY
    if chain_id == 1:
        url = 'https://api.etherscan.io/api'
    elif chain_id == 5:
        url = 'https://api-goerli.etherscan.io/api'
    params = {
        'module': 'contract',
        'action': 'getabi',
        'address': contract_address,
        'apikey': api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data['status'] != '1':
        print("Error: ", data['message'])
        return None
    else:
        abi = data['result']
        return abi

### Create account for signing transactions

In [7]:
account = Account.from_key(credentials.private_key)
print(account.address)
print(w3.from_wei(w3.eth.get_balance(account.address), 'ether'))

0x6B12B690Fc279958715C29AD65CCCB33900f0F0B
4.119310906115370405


### Simple Test

In [5]:
transaction = {
    'from': account.address,
    'to': account.address,
    'value': 50000000000000000,
    'nonce': w3.eth.get_transaction_count(account.address),
    'gas': 21000,
    'maxFeePerGas': 2000000000,
    'maxPriorityFeePerGas': 1500000000,
    'chainId': 5,
}
signed = w3.eth.account.sign_transaction(transaction, credentials.private_key)
tx_hash = w3.eth.send_raw_transaction(signed.rawTransaction)
tx = w3.eth.wait_for_transaction_receipt(tx_hash)
tx

AttributeDict({'blockHash': HexBytes('0xdf4fe7f96f593cb6394c422d91ca2e166bc3e17d07f82d8202ebd6e6e0f06b50'),
 'blockNumber': 9618858,
 'contractAddress': None,
 'cumulativeGasUsed': 1215350,
 'effectiveGasPrice': 1500000009,
 'from': '0x6B12B690Fc279958715C29AD65CCCB33900f0F0B',
 'gasUsed': 21000,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1,
 'to': '0x6B12B690Fc279958715C29AD65CCCB33900f0F0B',
 'transactionHash': HexBytes('0x54cf577d3a869a6419b492229ff7dc07ae10d87d7db29106

### Interact with Uniswap V2 Router

In [32]:
UNISWAP_V2_FACTORY = Web3.to_checksum_address("0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f")
UNISWAP_V2_ROUTER = Web3.to_checksum_address("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D")

UNISWAP_V2_FACTORY_ABI = get_abi(UNISWAP_V2_FACTORY, chain_id=5)
UNISWAP_V2_ROUTER_ABI = get_abi(UNISWAP_V2_ROUTER, chain_id=5)

In [37]:
UNISWAP_V2_FACTORY_CONTRACT = w3.eth.contract(address=UNISWAP_V2_FACTORY, abi=UNISWAP_V2_FACTORY_ABI)
UNISWAP_V2_ROUTER_CONTRACT = w3.eth.contract(address=UNISWAP_V2_ROUTER, abi=UNISWAP_V2_ROUTER_ABI)
# WETH = Web3.to_checksum_address("0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2") #mainnet WETH address
WETH = Web3.to_checksum_address(UNISWAP_V2_ROUTER_CONTRACT.functions.WETH().call())
UNI = Web3.to_checksum_address("0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984")
METH = Web3.to_checksum_address("0xdd69db25f6d620a7bad3023c5d32761d353d3de9")

WETH_abi = get_abi(WETH, chain_id=5)
WETH_contract = w3.eth.contract(address=WETH, abi=WETH_abi)
WETH_decimals = WETH_contract.functions.decimals().call()
WETH_symbol = WETH_contract.functions.symbol().call()

UNI_abi = get_abi(UNI, chain_id=5)
UNI_contract = w3.eth.contract(address=UNI, abi=UNI_abi)
UNI_decimals = UNI_contract.functions.decimals().call()
UNI_symbol = UNI_contract.functions.symbol().call()

METH_abi = get_abi(METH, chain_id=5)
METH_contract = w3.eth.contract(address=METH, abi=METH_abi)
METH_decimals = METH_contract.functions.decimals().call()
METH_symbol = METH_contract.functions.symbol().call()

'''
Buy Token:
swapExactETHForTokensSupportingFeeOnTransferTokens(uint256,address[],address,uint256)

Sell Token:
swapExactTokensForETHSupportingFeeOnTransferTokens(uint256,uint256,address[],address,uint256)

Rotate Token:
swapExactTokensForTokensSupportingFeeOnTransferTokens(uint256,uint256,address[],address,uint256)
'''

'\nBuy Token:\nswapExactETHForTokensSupportingFeeOnTransferTokens(uint256,address[],address,uint256)\n\nSell Token:\nswapExactTokensForETHSupportingFeeOnTransferTokens(uint256,uint256,address[],address,uint256)\n\nRotate Token:\nswapExactTokensForTokensSupportingFeeOnTransferTokens(uint256,uint256,address[],address,uint256)\n'

In [39]:
# Build and sign a transaction to buy token:
buy_tx = UNISWAP_V2_ROUTER_CONTRACT.functions.swapExactETHForTokensSupportingFeeOnTransferTokens(
    w3.to_wei(0.000006, 'ether'),    # amountOutMin
    [WETH, METH],                # path (path 0 needs to be WETH)
    account.address,            # to
    int(time.time()) + 60       # deadline (current + 60s)
).build_transaction({
    "from": account.address,
    "nonce": w3.eth.get_transaction_count(account.address),
    "value": w3.to_wei(0.1, 'ether'),
    "gas": 300000,
    "maxFeePerGas": 2000000000,
    "maxPriorityFeePerGas": 1500000000,
})

signed_buy_tx = w3.eth.account.sign_transaction(buy_tx, private_key=credentials.private_key)

# Send the raw transaction:
tx_hash = w3.eth.send_raw_transaction(signed_buy_tx.rawTransaction)
tx = w3.eth.wait_for_transaction_receipt(tx_hash)

if tx.status == 1:
    max_uint96 = (2 ** 96) - 1
    max_uint256 = (2 ** 256) - 1
    print("Transaction successful")
    #Auto Approve Token for selling
    approve_tx = METH_contract.functions.approve(UNISWAP_V2_ROUTER, max_uint256).build_transaction({
        "from": account.address,
        "nonce": w3.eth.get_transaction_count(account.address),
        "gas": 60000,
        "maxFeePerGas": 2000000000, 
        "maxPriorityFeePerGas": 1500000000,
    })
    #Auto Approve Token for selling if not approved
    allowance = METH_contract.functions.allowance(account.address, UNISWAP_V2_ROUTER).call()
    if allowance != max_uint256 or allowance != max_uint96:
        signed_approve_tx = w3.eth.account.sign_transaction(approve_tx, private_key=credentials.private_key)
        tx_hash = w3.eth.send_raw_transaction(signed_approve_tx.rawTransaction)
        tx = w3.eth.wait_for_transaction_receipt(tx_hash)
        print("Token approved")
    else:
        print("Token already approved")

Transaction successful
Token already approved


In [44]:
# Build and sign a transaction to sell token:
sell_tx = UNISWAP_V2_ROUTER_CONTRACT.functions.swapExactTokensForETHSupportingFeeOnTransferTokens(
    # w3.to_wei(0.000012, 'ether'),   # amountIn (amount of UNI to sell)
    METH_contract.functions.balanceOf(account.address).call(), # amountIn (sell all token in wallet)
    w3.to_wei(0.18, 'ether'),  # amountOutMin (Minimum amount of ETH to receive)
    [METH, WETH],                # path (path 1 needs to be WETH)
    account.address,            # to
    int(time.time()) + 60       # deadline (current + 60s)
).build_transaction({
    "from": account.address,
    "nonce": w3.eth.get_transaction_count(account.address),
    "gas": 300000,
    "maxFeePerGas": 2000000000,
    "maxPriorityFeePerGas": 1500000000,
})

signed_sell_tx = w3.eth.account.sign_transaction(sell_tx, private_key=credentials.private_key)
tx_hash = w3.eth.send_raw_transaction(signed_sell_tx.rawTransaction)
tx = w3.eth.wait_for_transaction_receipt(tx_hash)
if tx.status == 1:
    print("Transaction successful")

Transaction successful


### Price and Slipperage

In [13]:
'''
Given token address
Find token Pair
Get token price in WETH
'''
#Get token price in WETH
pair_address = UNISWAP_V2_FACTORY_CONTRACT.functions.getPair(UNI, WETH).call()
if w3.to_int(hexstr=pair_address) == 0:
    print("Pair not found")
    raise

pair_abi = get_abi(pair_address , chain_id=5)
pair_contract = w3.eth.contract(address=pair_address, abi=pair_abi)

In [14]:
print(f'Current Block number: {w3.eth.block_number}')
reserve = pair_contract.functions.getReserves().call()

#Caculate token pair rate (Token 1 is WETH)
if UNI < WETH:
    print("1")
    token_0_reserve = reserve[0]/10**UNI_decimals
    token_1_reserve = reserve[1]/10**WETH_decimals
else:
    print("2")
    token_0_reserve = reserve[1]/10**UNI_decimals
    token_1_reserve = reserve[0]/10**WETH_decimals
    
token_pair_rate = token_0_reserve/token_1_reserve

print(f"Token 0 ({UNI_symbol}) reserve: {token_0_reserve}")
print(f"Token 1 ({WETH_symbol}) reserve: {token_1_reserve}")
print(f"Block timestamp last: {reserve[2]}\n")

#Raw token rate
print(f"Raw token price: 1 {WETH_symbol} = {token_pair_rate} {UNI_symbol}")
print(f"Raw token price: 1 {UNI_symbol} = {1/token_pair_rate} {WETH_symbol}")

#AmountIn rate (Buy price)
if WETH < UNI:
    reserveOut = reserve[1]/10**UNI_decimals
    reserveIn = reserve[0]/10**WETH_decimals #Base token
else:
    reserveOut = reserve[0]/10**UNI_decimals
    reserveIn = reserve[1]/10**WETH_decimals #Base token

amountIn = 1 #1 WETH
amountIn = amountIn * 997/1000 #fee 0.3%
amountOut = amountIn * reserveOut/(reserveIn+amountIn)
print(f"True token price: Max amount {UNI_symbol} (out) given 1 {WETH_symbol} in: {amountOut}")

#AmountOut rate (Sell price)
if WETH < UNI:
    reserveIn = reserve[1]/10**UNI_decimals
    reserveOut = reserve[0]/10**WETH_decimals #Base token
else:
    reserveIn = reserve[0]/10**UNI_decimals
    reserveOut = reserve[1]/10**WETH_decimals #Base token

amountOut = 1 #1 WETH
amountOut = amountOut * 1000/997 #fee 0.3%
amountIn = amountOut * reserveIn / (reserveOut - amountOut)
print(f"True token price: Min amount {UNI_symbol} (in) given 1 {WETH_symbol} out: {amountIn}")

Current Block number: 9618907
1
Token 0 (UNI) reserve: 113538.69437312728
Token 1 (WETH) reserve: 5414.667924987667
Block timestamp last: 1693581036

Raw token price: 1 WETH = 20.96872715853316 UNI
Raw token price: 1 UNI = 0.04769006685239133 WETH
True token price: Max amount UNI (out) given 1 WETH in: 20.901972307724645
True token price: Min amount UNI (in) given 1 WETH out: 21.035719267279116
